<a href="https://colab.research.google.com/github/hc2twv/UPSE_OP/blob/main/Actividad6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 58.7 MB/s eta 0:00:00


Explicación del Código
Definir el Problema:

* Usamos pulp.LpProblem para definir nuestro problema de optimización, especificando que queremos minimizar el costo total.
Conjuntos y Parámetros:

* Definimos los nodos, distancias entre nodos, costos por km para cada tipo de enlace, el costo del equipo de red y el tráfico de datos.
Variables de Decisión:

* Creamos variables binarias
x,
y y
z usando pulp.LpVariable.dicts para indicar si se instala un enlace de 10 Mbps, 20 Mbps o 40 Mbps respectivamente entre las sucursales.
Función Objetivo:

* Definimos la función objetivo para minimizar el costo total de instalación de los enlaces y los equipos de red.
Restricciones:

* Añadimos restricciones para asegurar que se satisfaga el volumen de tráfico de datos, que solo se instale un tipo de enlace entre cualquier par de sucursales y que cada sucursal tenga al menos un equipo de red.
Resolver el Problema:

* Utilizamos prob.solve() para resolver el problema de optimización.

In [4]:
import pulp

# Definir el problema de optimización
prob = pulp.LpProblem("Telecom_Network", pulp.LpMinimize)

# Conjuntos de nodos
nodes = ['A', 'B', 'C']

# Distancias entre nodos
distances = {
    ('A', 'B'): 100,
    ('A', 'C'): 200,
    ('B', 'C'): 150
}

# Costos por km para cada tipo de enlace
cost_per_km = {
    '10Mbps': 100,
    '20Mbps': 150,
    '40Mbps': 200
}

# Costo de equipo de red por nodo
cost_node = 500

# Volumen de tráfico de datos
traffic = {
    ('A', 'B'): 10,
    ('A', 'C'): 20,
    ('B', 'C'): 15
}

# Variables de decisión
x = pulp.LpVariable.dicts("x", [(i, j) for i in nodes for j in nodes if i < j], 0, 1, pulp.LpBinary)
y = pulp.LpVariable.dicts("y", [(i, j) for i in nodes for j in nodes if i < j], 0, 1, pulp.LpBinary)
z = pulp.LpVariable.dicts("z", [(i, j) for i in nodes for j in nodes if i < j], 0, 1, pulp.LpBinary)

# Función objetivo: minimizar el costo total de la red
prob += pulp.lpSum([
    distances[i, j] * cost_per_km['10Mbps'] * x[i, j] +
    distances[i, j] * cost_per_km['20Mbps'] * y[i, j] +
    distances[i, j] * cost_per_km['40Mbps'] * z[i, j]
    for (i, j) in distances
]) + cost_node * len(nodes), "Costo_Total"

# Restricción: asegurar el volumen de tráfico de datos
prob += pulp.lpSum([
    10 * x[i, j] + 20 * y[i, j] + 40 * z[i, j]
    for (i, j) in distances
]) >= pulp.lpSum([traffic[i, j] for (i, j) in traffic]), "Volumen_de_Trafico"

# Restricción: solo un tipo de enlace entre cualquier par de sucursales
for (i, j) in distances:
    prob += x[i, j] + y[i, j] + z[i, j] <= 1, f"Un_Solo_Enlace_{i}_{j}"

# Restricción: cada sucursal debe tener un equipo de red
for i in nodes:
    prob += pulp.lpSum([
        x[i, j] + y[i, j] + z[i, j] if i < j else x[j, i] + y[j, i] + z[j, i]
        for j in nodes if i != j
    ]) >= 1, f"Equipo_de_Red_{i}"

# Resolver el problema
prob.solve()

# Resultados
print(f"Estado: {pulp.LpStatus[prob.status]}")
print(f"Costo Total: ${pulp.value(prob.objective):,.2f}")

for (i, j) in distances:
    if pulp.value(x[i, j]) == 1:
        print(f"Enlace 10 Mbps entre {i} y {j}")
    elif pulp.value(y[i, j]) == 1:
        print(f"Enlace 20 Mbps entre {i} y {j}")
    elif pulp.value(z[i, j]) == 1:
        print(f"Enlace 40 Mbps entre {i} y {j}")


Estado: Optimal
Costo Total: $36,500.00
Enlace 40 Mbps entre A y B
Enlace 10 Mbps entre B y C


ejercicio 2

In [20]:
import pulp

# Definir el problema de optimización
prob = pulp.LpProblem("Maximize_Flow", pulp.LpMaximize)

# Parámetros del problema
distance = 150  # km
cost_per_km = {
    '10Mbps': 100,
    '20Mbps': 150,
    '40Mbps': 200
}
cost_node = 500  # por nodo
budget = 2000  # presupuesto total

# Variables de decisión
x_10 = pulp.LpVariable("x_10", 0, 1, pulp.LpBinary)
x_20 = pulp.LpVariable("x_20", 0, 1, pulp.LpBinary)
x_40 = pulp.LpVariable("x_40", 0, 1, pulp.LpBinary)

# Función objetivo: maximizar el flujo de datos
prob += 10 * x_10 + 20 * x_20 + 40 * x_40, "Max_Flow"

# Restricción: presupuesto total
prob += (cost_per_km['10Mbps'] * x_10 +
         cost_per_km['20Mbps'] * x_20 +
         cost_per_km['40Mbps'] * x_40 +
         2 * cost_node) <= budget, "Budget_Constraint"

# Resolver el problema
prob.solve()

# Resultados
print(f"Estado: {pulp.LpStatus[prob.status]}")
print(f"Flujo Máximo: {pulp.value(prob.objective)} Mbps")

if pulp.value(x_10) == 1:
    print("Enlace seleccionado: 10 Mbps")
elif pulp.value(x_20) == 1:
    print("Enlace seleccionado: 20 Mbps")
elif pulp.value(x_40) == 1:
    print("Enlace seleccionado: 40 Mbps")
else:
    print("No se seleccionó ningún enlace dentro del presupuesto.")


Estado: Optimal
Flujo Máximo: 70.0 Mbps
Enlace seleccionado: 10 Mbps
